This notebook will walk you through: 
1. Importing an Amazon review dataset into a PostgreSQL database.
2. Setting up a model that has the objective of returning a personalized list of the recommended products on a home or discovery page.
3. Fetching ranked products for a specific user!

Let's get started! 🚀

### Setup

Replace `YOUR_API_KEY` with your API key below.

*If you don't have an API Key, feel free to [signup on our website](https://www.shaped.ai/#contact-us) :)*

In [ ]:
SHAPED_BASE_URL = "https://api.prod.shaped.ai/v0"
SHAPED_API_KEY = YOUR_API_KEY

Install the packages needed:
- `requests` is needed for making HTTP requests
- `pandas` is needed for handling the data
- `ipython-sql` is needed for connecting with the database
- `sqlalchemy` is needed for executing db queries via DBApi's
- `psycopg2` is needed for postgresql connection

In [ ]:
%%capture
!pip install requests
!pip install pandas
!pip install sqlalchemy
!pip install psycopg2
!pip install boto3

In [ ]:
import requests
import pandas as pd
import json
import gzip
import csv
import boto3
from io import StringIO
from sqlalchemy import create_engine

### Preview Dataset

[The Amazon dataset](http://jmcauley.ucsd.edu/data/amazon/links.html) has a lot of data! Looking through it, we want the interaction and item data so we'll be using a small subset of [ratings only](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Grocery_and_Gourmet_Food.csv). Note that those contain a smaller subset of the full amazon dataset, we chose small files to keep this notebook runtime low but could also run it with the bigger ones.

For the interaction data we take a look at the `Sample review` section on the website and see the relevant columns for the interactions are:
- `reviewerID`: Is the user who is reviewing the item.
- `asin`: Is a unique identification for a product. It will be used as an item to train our models.
- `overall`: Is the review of an product given by a user.
- `unixReviewTime`: Is the review of an product given by a user.



We will download the data and extract to a Pandas DataFrame and then upload that dataframe to a Postgres DB. 


In [ ]:
ratings_df = pd.read_csv("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Grocery_and_Gourmet_Food.csv", names=["reviewer_id", "asin", "overall", "review_time"], header=None)
ratings_df.head()


Next, you could download the metadata file to create the items dataframe uncommenting the next cell that comes from the script provided on the website (Note this took ~11 hours on an M1 Pro with 16GB of RAM). We are leaving this as an optional step due to its runtime.

Download the metadata file [here](http://snap.stanford.edu/data/amazon/productGraph/metadata.json.gz) and extract it inside this notebook path. Change the `using_metadata` variable to True.

In [ ]:
using_metadata = False

In [ ]:
if using_metadata:
    def parse(path):
        g = gzip.open(path, 'rb')
        for l in g:
            yield eval(l)

    def getDF(path):
        i = 0
        df = {}
        for d in parse(path):
            df[i] = d
            i += 1
        return pd.DataFrame.from_dict(df, orient='index')

    item_df = getDF("metadata.json.gz")
    item_df = item_df[["asin", "title", "price"]]
    item_df

### Upload Dataset to Postgres
To make the upload quicker we'll drop the unnecessary columns from the items dataframe:

Now that we have the two dataframes, we'll upload them to a Postgres DB. First we need to setup a Postgres database. We'll use boto3 to create one in AWS RDS, but you can host it anywhere.

In [ ]:
rds = boto3.client('rds', aws_access_key_id='AWS-ACCESS-KEY-ID-OF-YOUR-AWS-ACCOUNT',
                     aws_secret_access_key='AWS-SECRET-ACCESS-KEY-OF-YOUR-AWS-ACCOUNT',
                     region_name='us-east-2')

db_name = "amazon_ratings"
db_instance_name="shaped-demo"
username="your_username"
password="your_password"
port=5432

response = rds.create_db_instance(
    AllocatedStorage=10,
    DBName=db_name,
    DBInstanceIdentifier=db_instance_name,
    DBInstanceClass="db.t2.micro",
    Engine="postgres",
    EngineVersion="12",
    MasterUsername=username,
    MasterUserPassword=password,
    Port=port,
)

We wait until its state changes from `creating` and get its address

In [ ]:
response = rds.describe_db_instances()
found = False
while not found: 
    for instance in response["DBInstances"]:
        if instance["DBInstanceIdentifier"] == db_instance_name and instance["DBInstanceStatus"]!="creating":
            db_endpoint = instance["Endpoint"]["Address"]

            print(f"Endpoint with status '{instance['DBInstanceStatus']}' at address {db_endpoint}")
            found=True
            break

Then we'll connect to the database, create a table for each of the data frames, and upload the respective data.

In [ ]:
def psql_insert_copy(table, conn, keys, data_iter):
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(f"postgresql://{username}:{password}@{db_endpoint}:{port}/{db_name}")
ratings_df.to_sql('ratings', engine, method=psql_insert_copy, index=False)

if using_metadata:
    item_df.to_sql('products', engine, method=psql_insert_copy, index=False) # Uncoment if you want to upload the items df

To confirm the data was uploaded correctly, let's run a couple of queries.

In [ ]:
reviews_df = pd.read_sql('select * from ratings limit 5', engine)
reviews_df

In [ ]:
# Uncomment if you added the items df
if using_metadata:
    item_df = pd.read_sql('select * from products limit 5', engine)
    item_df


### Setup Endpoint

Once we have all our data prepared, we can upload it to Shaped using a [`POST` call to the `/models` endpoint](https://docs.shaped.ai/reference/create-model). The request body contains all the info needed to setup the model. If successful, the response body contains a uri that can be used to fetch details about the model. Otherwise it returns a detailed error message with guidance on what needs to be fixed.

*If you try `POST`ing to the `/models` endpoint multiple times with the same `model_name`, you will encounter an error saying `"Model with name: '{model_name}' already exists with status: '{status}'"`. If you would like to update or create a new model with the same `model_name` you must first delete the existing model with `model_name`. You can do that by making a [`DELETE` request to the `/models/{model_name}` endpoint](https://docs.shaped.ai/reference/delete-model). The `DELETE` call can be made from the cell in the Clean Up section at the bottom of this notebook.*

In [ ]:
# Item payload on our request will be different if we are using features from the metadata file or not.
if using_metadata:
    item_payload = {
      "id": "product_id",
      "source": {
        "connector_id": "postgres_amazon_ratings",
        "query": "select asin as product_id, title, price from products limit 10000"
      },
      "features": [
        {
          "name":"title",
          "type":"Text"
        },
        {
          "name":"price",
          "type":"Numerical"
        }
      ]
    }
else:
    item_payload = {
      "id": "product_id",
    }

In [ ]:
model_name = "amazon_dataset_postgres"

payload = json.dumps({
  "connector_configs": [{
    "id": "postgres_amazon_ratings",
    "type": "Postgres",
    "user": username,
    "password": password,
    "host": db_endpoint,
    "port": port,
    "database": db_name
  }],
  "model_name": model_name,
  "schema": {
    "user": {
      "id": "reviewer_id",
    },
    "item": item_payload,
    "interaction": {
      "created_at": "review_time",
      "label": {
        "name": "overall",
        "type": "Rating"
      },
      "source": {
        "connector_id": "postgres_amazon_ratings",
        "query": "select reviewer_id, asin as product_id, overall::float, review_time from ratings limit 1000000"
      }
    }
  }
})

headers = {
  'x-api-key': SHAPED_API_KEY,
  'Content-Type': 'application/json'
}

response = requests.request("POST", f"{SHAPED_BASE_URL}/models", headers=headers, data=payload)

print(response.status_code)
assert response.status_code==200
print(json.dumps(json.loads(response.content), indent=2))

### Rank!

After we make the `POST` call to `/models`, we can make a [`GET` call to `/models`](https://docs.shaped.ai/reference/list-models) to see our newly created model. 

In [ ]:
response = requests.get(
    f"{SHAPED_BASE_URL}/models",
    headers={
        "x-api-key": SHAPED_API_KEY,
        "Content-Type":"application/json"
    }
)
print(json.dumps(json.loads(response.content), indent=2))

You'll notice the `"status"` of the model you just created is most likely `"SCHEDULING"`. This means that the initial training job hasn't completed yet. The amount of time it takes will be dependent on the amount of data. Feel free to keep querying the `/models` endpoint to check the status of your model. When it is ready, the `"status"` will be `"ACTIVE"`. The different states are listed in detail on the [model details endpoint](https://docs.shaped.ai/reference/model-details).

Once your model is ready, (`"status": "ACTIVE"`), you can hit the [rank endpoint](https://docs.shaped.ai/reference/rank)!

Remember, `user_id` is the id of the User you want to fetch rankings for. You can also add an optional query param, `limit`, which will inform how many results to return (with the default being 15).

In [ ]:
response = requests.get(
    f"{SHAPED_BASE_URL}/models/{model_name}/rank?user_id=1",
    headers={
        "x-api-key": SHAPED_API_KEY,
        "Content-Type":"application/json"
    }
)
print(json.dumps(json.loads(response.content), indent=2))

Wow! It was that easy to see the top ranked items for the passed in `user_id` 🍾. Now let's add ranking to your product :)

### Clean Up

__The below code should ONLY be run if you want to delete the model with `model_name`.__

In [ ]:
response = requests.delete(
    f"{SHAPED_BASE_URL}/models/{model_name}",
    headers={
        "x-api-key": SHAPED_API_KEY,
        "Content-Type":"application/json"
    }
)
print(json.dumps(json.loads(response.content), indent=2))